In [ ]:
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
# plt.ylim(0, 1)
plt.legend(['Training data', 'Validation data'], loc='lower right')
plt.show()

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
# plt.ylim(0, 1)
plt.legend(['Training data', 'Validation data'], loc='lower right')
plt.show()

In [ ]:
score = model.evaluate(x_test, y_test)

In [ ]:
x = np.load(PATH_TEST_DATA+"2303"+ ".npy" ).transpose()
y = np.load(PATH_TEST_LABELS+"2303"+ ".npy" )

prediction = model.predict(x).round().transpose()
truth = y.transpose()

print(prediction.shape)
print(truth.shape)

end_seconds = 100
end_index = round(end_seconds/(0.032))

plt.figure()
plt.subplot()
plt.imshow(truth[:,0:end_index],cmap='Greys',aspect='auto')

plt.figure()
plt.subplot()
plt.imshow(prediction[:,0:end_index],cmap='Greys',aspect='auto')

In [ ]:
def post_process(y, min_rest_len, min_note_len = -1):
    for i in range(y.shape[1]):
        rest_count = 0
        note_count = 0
        for j in range(y.shape[0]):
            #If the preceeding and the current event is a rest, keep counting rests
            if y[j,i] == 0 and (counting_rests == True or j+1<min_rest_len): 
                counting_rests = True
                rest_count = rest_count + 1
            #If the preceeding and the current event is a note, keep counting notes
            elif y[j,i] == 1 and (counting_rests ==False or j+1<min_note_len):
                counting_rests = False
                note_count = note_count +1
            elif y[j,i] == 0 : 
                counting_rests = True
                #If it is a first rest, and the amount of notes preceeding it is too little
                if note_count < min_note_len:
                    y[j-note_count:j,i] = 0
                    rest_count = rest_count+ note_count +1
                    note_count = 0
                #If it is a first rest, and the amount of notes preceeding it is long enough
                else:
                    rest_count = 1
            elif y[j,i] == 1:
                counting_rests = False
                #If it is a note note, and the amount of rests preceeding it is too little
                if rest_count < min_rest_len:
                    y[j-rest_count:j,i] = 1
                    note_count = note_count+ rest_count+1
                    rest_count = 0
                #If it is a first note, and the amount of rests preceeding it is long enough
                else:
                    note_count = 1
    return y
